# Análisis de Aspiradoras

El Objetivo de esta Note es los siguientes:

1. Obtener información de la página de Amazon sobre aspiradoras
2. Guardar las información de las 200 primeras aspiradoras en un dataframe
3. Pasar el DataFrame y aplicarle técnicas de aprendizaje No Supervisado para clasificar las aspiradoras

## Obtener Información de Aspiradoras de Amazon

### Web Scraping

#### BeautifulSoup

Es el primer método que vamos a utilizar para poder extraer datos de la Web

In [1]:
#Líbrerias
import requests

#url = 'https://www.amazon.com/'
url = 'https://www.amazon.es/s?k=aspiradoras&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_2'

aspiradoraWeb = requests.get(url)
aspiradoraWeb.status_code

503

* El primer método de Scraping iba hacer con BeautifulSoup, sin embargo, tengo un inconveniente debido a que el status code me da 503, eso significa que no esta accesible el servidor, esto puede deberse a que hay páginas que no permiten realizar el WebScrapper por la líbreria requests

* Para resolver este caso vamos a utilizar Selenium

#### Selenium

##### 1. Configuracion y parametrización de Selenium

In [2]:
#Importamos Selenium
from selenium import webdriver

#Parametrizamos el driver
options = webdriver.ChromeOptions()
#Para navegar en modo incognito
options.add_argument('--incognito')

#Abrimos el driver
driver = webdriver.Chrome(executable_path='c:/chromedriver.exe',options=options)

#Nos vamos a la URL
driver.get(url)

##### 2. Obtener los links de cada una de las aspiradoras que se encuentran en una página y lo almaceno en una lista

In [ ]:
# Vamos a obtener los links y los nombres de las aspiradoras
aspiradoras = driver.find_elements_by_xpath('//a[@class="a-link-normal a-text-normal"]')

#Número de aspiradoras 
len(aspiradoras)

In [ ]:
#vamos a obtener los links de cada aspiradora
linksAspiradoras = []
aspiradora = aspiradoras[0]

linkAspiradora = aspiradora.get_attribute('href')
linkAspiradora

#Obtengo los links de las aspiradoras
for aspiradora in aspiradoras:
    linksAspiradoras.append(aspiradora.get_attribute('href'))
    
linksAspiradoras

In [ ]:
# Se cierra el driver
driver.close()

##### 3. Vamos a obtener la información de cada aspiradora:

    * Nombre
    * Precio
    * Número de Opiniones
    * Número de estrellas
    * Tabla de detalles técnicos

In [ ]:
# Vamos a obtener los datos de cada aspiradora
driver = webdriver.Chrome(executable_path='c:/chromedriver.exe',options=options)
# Obtengo el primer link del links de aspiradoras
driver.get(linksAspiradoras[3])

In [ ]:
#Se obtiene el nombre de la aspiradora
nombreAspiradora = driver.find_element_by_xpath('//*[@id="productTitle"]').text
#Se obtiene el número de opiniones
numeroOpinion = driver.find_element_by_xpath('//*[@id="acrCustomerReviewText"]').text
#Se obtiene el precio
precio = driver.find_element_by_xpath('//td[@class = "a-span12"]/span[contains(@id, "priceblock_")]').text
#Se obtiene el numero de estrellas
#estrella = driver.find_element_by_xpath('//i[contains(@class,"a-icon a-icon-star")]/span[@class="a-icon-alt"]').text
estrella = driver.find_element_by_xpath('//a[@class="a-popover-trigger a-declarative"]/i[contains(@class,"a-icon a-icon-star")]/span[@class="a-icon-alt"]')




print(nombreAspiradora)
print(numeroOpinion)
print(precio)
print(estrella.text)
#len(estrella)

In [ ]:
#Obteniendo la tabla de caracteristicas técnicas
#detallesTecnicos = driver.find_elements_by_xpath('//*[@id="prodDetails"]/div[2]/div[1]/div[1]/div[2]/div/div/table/tbody/tr')detallesTecnicossTecnicos = driver.find_elements_by_xpath('//')
detallesTecnicos = driver.find_elements_by_xpath('//div[@class="pdTab"]//tbody/tr')

detallesTecnicos


In [ ]:
if len(detallesTecnicos)>0:
    # Obtengo el primer detalle técnico de la table en este caso la etiqueta
    detallesTecnicos[0].find_element_by_xpath('.//td[@class="label"]').text
    # Obtengo el primer detalle técnico de la table en este caso el valor
    detallesTecnicos[0].find_element_by_xpath('.//td[@class="value"]').text

    tablaDetalleTecnico = {}

    for detalle in detallesTecnicos:
        try:
            etiqueta = detalle.find_element_by_xpath('.//td[@class="label"]').text
            valor = detalle.find_element_by_xpath('.//td[@class="value"]').text

            # El valor le paso como una lista para que sea mas fácil la conversion a Dataframe
            tablaDetalleTecnico.update({etiqueta:[valor]})
        except:
            etiqueta = ''
            valor = ''    


    tablaDetalleTecnico
else:
    tablaDetalleTecnico = {}

##### 4. Pasar a un Dataframe los datos

In [ ]:
#Mandando a un dataframe los detalles tecnicos

import pandas as pd

tablaDetalleTecnico

DatoAspiradora = pd.DataFrame(tablaDetalleTecnico)
DatoAspiradora

In [ ]:
print(nombreAspiradora)
print(numeroOpinion)
print(precio)

In [ ]:
# Vamos añadir las columnas de:
# Nombre de la aspiradora; las valoraciones; y el precio en el DataFrame
DatoAspiradora['Nombre Aspiradora'] = nombreAspiradora
DatoAspiradora['Opiniones'] = numeroOpinion
DatoAspiradora['Precio'] = precio

len(DatoAspiradora)

##### 5. Obtenemos solo los datos que necesito para el Análisis

    * Nombre de la aspiradora
    * Precio
    * Número de Opiniones
    * Peso del producto
    * Potencia
    * Dimensiones del producto
    * Volúmen

In [ ]:
#Veo las columnas que tiene el Dataframe
DatoAspiradora.columns

In [ ]:
driver.close()

##### 6. Obtener todos los datos de las 200 aspiradoras

###### Librerias

In [170]:
#Librerias
import pandas as pd
import time
from selenium import webdriver

###### Paginas en donde se encuentran las aspiradoras

In [171]:
#Todas las paginas de aspiradoras
paginaAspiradoras = [
    'https://www.amazon.es/s?k=aspiradoras&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_2',
    'https://www.amazon.es/s?k=aspiradoras&page=2&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583681798&ref=sr_pg_2',
    'https://www.amazon.es/s?k=aspiradoras&page=3&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583681876&ref=sr_pg_3',
    'https://www.amazon.es/s?k=aspiradoras&page=4&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583681902&ref=sr_pg_4',
    'https://www.amazon.es/s?k=aspiradoras&page=5&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583681936&ref=sr_pg_5',
    'https://www.amazon.es/s?k=aspiradoras&page=6&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583681959&ref=sr_pg_6',
    'https://www.amazon.es/s?k=aspiradoras&page=7&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583681982&ref=sr_pg_7',
    'https://www.amazon.es/s?k=aspiradoras&page=8&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583682002&ref=sr_pg_8',
    'https://www.amazon.es/s?k=aspiradoras&page=9&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583682017&ref=sr_pg_9',
    'https://www.amazon.es/s?k=aspiradoras&page=10&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583687663&ref=sr_pg_10',
    'https://www.amazon.es/s?k=aspiradoras&page=11&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583687688&ref=sr_pg_11'
]

paginaAspiradoras

['https://www.amazon.es/s?k=aspiradoras&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_2',
 'https://www.amazon.es/s?k=aspiradoras&page=2&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583681798&ref=sr_pg_2',
 'https://www.amazon.es/s?k=aspiradoras&page=3&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583681876&ref=sr_pg_3',
 'https://www.amazon.es/s?k=aspiradoras&page=4&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583681902&ref=sr_pg_4',
 'https://www.amazon.es/s?k=aspiradoras&page=5&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583681936&ref=sr_pg_5',
 'https://www.amazon.es/s?k=aspiradoras&page=6&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583681959&ref=sr_pg_6',
 'https://www.amazon.es/s?k=aspiradoras&page=7&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583681982&ref=sr_pg_7',
 'https://www.amazon.es/s?k=aspiradoras&page=8&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1583682002&ref=sr_pg_8',
 'https://www.amazon.es/s?k=aspiradoras&page=9&__mk_es_ES=%C3%85M

###### Función que obtiene los links de cada aspiradora por cada una de las páginas

In [176]:
# Funcion para obtener todos los links de las apiradoras pasandole la URL principla de la pagina
def obtenerLinksAspiradoras(listaPagina):
    
    linksAspiradoras = []
    
    for i, url in enumerate(listaPagina):
        
        print('Obteniendo links: {}/{}'.format(i+1,len(listaPagina)))
    
        options = webdriver.ChromeOptions()
        options.add_argument('--incognito')
        driver = webdriver.Chrome(executable_path='c:/chromedriver.exe',options=options)
        driver.get(url)
        
        # Vamos a obtener los links y los nombres de las aspiradoras
        aspiradoras = driver.find_elements_by_xpath('//a[@class="a-link-normal a-text-normal"]')
        
        #Obtengo los links de las aspiradoras
        for aspiradora in aspiradoras:
            linksAspiradoras.append(aspiradora.get_attribute('href'))
            
        driver.close()

        
    return linksAspiradoras
        
    
    

###### Función que obtiene información de cada aspiradora 

In [177]:
#Vamos a realizar un función en donde ingreso la URL de la aspiradora
# Y me devuelve un DataFrame con:
# El nombre de la aspiradora, precio,opiniones y detalles técnicos

def obtenerInformacionAspiradora(url):
    
    aspiradora = pd.DataFrame()
    tablaDetalleTecnico = {}
    
    driver = webdriver.Chrome(executable_path='c:/chromedriver.exe',options=options)
    # Obtengo el primer link del links de aspiradoras
    driver.get(url)

    #time.sleep(10)
    
    try:
    
        nombreAspiradora = driver.find_element_by_xpath('//*[@id="productTitle"]').text
        numeroOpinion = driver.find_element_by_xpath('//*[@id="acrCustomerReviewText"]').text
        precio = driver.find_element_by_xpath('//td[@class = "a-span12"]/span[contains(@id, "priceblock_")]').text
        detallesTecnicos = driver.find_elements_by_xpath('//div[@class="pdTab"]//tbody/tr')

        if len(detallesTecnicos)>0:
            # Obtengo el primer detalle técnico de la table en este caso la etiqueta
            detallesTecnicos[0].find_element_by_xpath('.//td[@class="label"]').text
            # Obtengo el primer detalle técnico de la table en este caso el valor
            detallesTecnicos[0].find_element_by_xpath('.//td[@class="value"]').text

            tablaDetalleTecnico = {}

            for detalle in detallesTecnicos:
                try:
                    etiqueta = detalle.find_element_by_xpath('.//td[@class="label"]').text
                    valor = detalle.find_element_by_xpath('.//td[@class="value"]').text

                    # El valor le paso como una lista para que sea mas fácil la conversion a Dataframe
                    tablaDetalleTecnico.update({etiqueta:[valor]})
                except:
                    etiqueta = ''
                    valor = ''    


            tablaDetalleTecnico
        else:
            tablaDetalleTecnico = {}

        DatoAspiradora = pd.DataFrame(tablaDetalleTecnico)
        DatoAspiradora['Nombre Aspiradora'] = nombreAspiradora
        DatoAspiradora['Opiniones'] = numeroOpinion
        DatoAspiradora['Precio'] = precio
        DatoAspiradora['URL'] = url
        

        aspiradora = DatoAspiradora

        driver.close()

        return aspiradora
    
    except:
        driver.close()

    
 

###### Ejecución del programa

* Obteniendo todos los links de las aspiradoras por cada página

In [178]:
# Obterner los links de las aspiradoras
links_apiradora = obtenerLinksAspiradoras(paginaAspiradoras)


Obteniendo links: 1/11
Obteniendo links: 2/11
Obteniendo links: 3/11
Obteniendo links: 4/11
Obteniendo links: 5/11
Obteniendo links: 6/11
Obteniendo links: 7/11
Obteniendo links: 8/11
Obteniendo links: 9/11
Obteniendo links: 10/11
Obteniendo links: 11/11


* Número de aspiradoras que se tiene

In [179]:
print('El numero de aspiradoras son:{}'.format(len(links_apiradora)))

El numero de aspiradoras son:236


* Obteniendo información de las aspiradoras una a una

In [180]:
# Obtiene todos los valores de las aspiradoras
datoAspiradorasX = pd.DataFrame()

for i,link in enumerate(links_apiradora):
    try:
        informacion = obtenerInformacionAspiradora(link)

        if len(informacion):
            print('Scrapeada:{}/{}'.format(i+1,len(links_apiradora)))
            datoAspiradorasX = datoAspiradorasX.append(informacion,sort=False,ignore_index=True)
        else:
            print('Sin dato de aspiradora')
            #datoAspiradorasX
    except:
        print('Hubo error en el metodo')
        
datoAspiradorasX

Scrapeada:1/236
Scrapeada:2/236
Scrapeada:3/236
Scrapeada:4/236
Scrapeada:5/236
Scrapeada:6/236
Scrapeada:7/236
Scrapeada:8/236
Scrapeada:9/236
Scrapeada:10/236
Scrapeada:11/236
Scrapeada:12/236
Scrapeada:13/236
Scrapeada:14/236
Scrapeada:15/236
Scrapeada:16/236
Scrapeada:17/236
Scrapeada:18/236
Sin dato de aspiradora
Scrapeada:20/236
Scrapeada:21/236
Scrapeada:22/236
Scrapeada:23/236
Scrapeada:24/236
Scrapeada:25/236
Scrapeada:26/236
Scrapeada:27/236
Sin dato de aspiradora
Scrapeada:29/236
Scrapeada:30/236
Scrapeada:31/236
Hubo error en el metodo
Scrapeada:33/236
Scrapeada:34/236
Scrapeada:35/236
Scrapeada:36/236
Sin dato de aspiradora
Scrapeada:38/236
Scrapeada:39/236
Sin dato de aspiradora
Scrapeada:41/236
Scrapeada:42/236
Scrapeada:43/236
Scrapeada:44/236
Scrapeada:45/236
Scrapeada:46/236
Scrapeada:47/236
Scrapeada:48/236
Scrapeada:49/236
Scrapeada:50/236
Scrapeada:51/236
Scrapeada:52/236
Scrapeada:53/236
Scrapeada:54/236
Scrapeada:55/236
Scrapeada:56/236
Scrapeada:57/236
Scrapeada

,Marca,Número de modelo,Peso del producto,Dimensiones del producto,Capacidad,Volumen,Potencia,Radio de acción,Eficiencia energética,Consumo anual,...,Salida de vapor,Tipo(s) de material,Método de instalación,Tipo de sujección de accesorio,Estilo,Amperaje,Patrón,Longitud del cable,Sistema de medida,Número de pieza del fabricante
0,AmazonBasics,15KC-71EU4,"4,5 Kg","36,3 x 29,4 x 24,7 cm",1.5 litros,1.5 litros,700 vatios,6 metros,A,23.4 kilowatt_hours_per_year,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,INSE,S6-EU,"2,69 Kg","34,5 x 14 x 24 cm",1.20 litros,NaN,250 vatios,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MEIYOU,NaN,"1,96 Kg",NaN,NaN,NaN,600 vatios,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Rowenta,RO2727EA,"4,95 Kg","47 x 28,5 x 30 cm",1.2 litros,1.2 litros,750 vatios,12 metros,NaN,28 kilowatt_hours,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rowenta,RO3753,"6,6 Kg","40,9 x 27,6 x 29 cm",1.5 litros,1.5 litros,750 vatios,6.2 metros,NaN,28 kilowatt_hours,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Cecotec,Conga EcoExtreme 3000,"4,5 Kg",41 x 27 x 31 cm,3.5 litros,NaN,700 vatios,12 metros,A+++,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Kärcher,1.629-800.0,"5,8 Kg","34 x 38,8 x 50,3 cm",17 litros,17 litros,1000 vatios,4 metros,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Cecotec,05029,6 Kg,40 x 40 x 52 cm,15 litros,15 litros,1400 vatios,7 metros,A,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Deenkee,NaN,"3,82 Kg",NaN,600 ml,NaN,1000 vatios,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Holife,NaN,"2,16 Kg","14 x 15,5 x 64 cm",0.8 litros,0.8 litros,600 vatios,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
#Se guardan los datos de las aspiradoras
datoAspiradorasX.to_csv('DatoAspiradoras.csv');
datoAspiradorasX.to_excel('DatoAspiradoras.xlsx');

In [186]:
#Datos que se van a utilizar
datoAspiradorasX.columns

Index(['Marca', 'Número de modelo', 'Peso del producto',
       'Dimensiones del producto', 'Capacidad', 'Volumen', 'Potencia',
       'Radio de acción', 'Eficiencia energética', 'Consumo anual',
       'Nivel de limpieza en alfombra', 'Ruido', 'ASIN',
       'Valoración media de los clientes',
       'Clasificación en los más vendidos de Amazon',
       'Producto en Amazon.es desde', 'Nombre Aspiradora', 'Opiniones',
       'Precio', 'URL', 'Dimensiones del paquete', 'Color',
       'Apagado automático', 'Características especiales', 'Diámetro',
       'Voltaje', 'Material', 'Nivel de limpieza en suelo',
       'Clase de reemisión de polvo', 'Número de velocidades',
       'Tiempo de funcionamiento', 'Identificador de producto del fabricante',
       'Número de modelo del producto', 'Tamaño', 'Fuente de alimentación',
       'Potencia eléctrica', 'Número de productos', 'Número de piezas',
       'Sonido', 'Componentes incluidos', 'Incluye baterías',
       'Necesita baterías', 'Peso',

##### 7. Limpieza de Datos

In [18]:
# Cargamos el Dataset creado para no volver a correr las funciones

import pandas as pd

datosAspiradoras = pd.read_csv('DatoAspiradoras.csv')
datosAspiradoras.head()

,Unnamed: 0,Marca,Número de modelo,Peso del producto,Dimensiones del producto,Capacidad,Volumen,Potencia,Radio de acción,Eficiencia energética,...,Salida de vapor,Tipo(s) de material,Método de instalación,Tipo de sujección de accesorio,Estilo,Amperaje,Patrón,Longitud del cable,Sistema de medida,Número de pieza del fabricante
0,0,AmazonBasics,15KC-71EU4,"4,5 Kg","36,3 x 29,4 x 24,7 cm",1.5 litros,1.5 litros,700 vatios,6 metros,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,INSE,S6-EU,"2,69 Kg","34,5 x 14 x 24 cm",1.20 litros,NaN,250 vatios,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,MEIYOU,NaN,"1,96 Kg",NaN,NaN,NaN,600 vatios,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Rowenta,RO2727EA,"4,95 Kg","47 x 28,5 x 30 cm",1.2 litros,1.2 litros,750 vatios,12 metros,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Rowenta,RO3753,"6,6 Kg","40,9 x 27,6 x 29 cm",1.5 litros,1.5 litros,750 vatios,6.2 metros,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Nuevo dato de aspiradoras
aspiradoras = datosAspiradoras[['Nombre Aspiradora','Precio','Opiniones','Peso','Potencia','Dimensiones del producto','Volumen']]
aspiradoras.head()

,Nombre Aspiradora,Precio,Opiniones,Peso,Potencia,Dimensiones del producto,Volumen
0,AmazonBasics - Aspiradora cilíndrica sin bolsa...,"49,99 €",3.934 valoraciones,NaN,700 vatios,"36,3 x 29,4 x 24,7 cm",1.5 litros
1,"INSE Aspiradora Escopa sin Cable, Vertical y d...","199,00 €",49 valoraciones,NaN,250 vatios,"34,5 x 14 x 24 cm",NaN
2,"Aspiradora Escoba, MEIYOU Aspirador Mano con C...","59,96 €",354 valoraciones,NaN,600 vatios,NaN,NaN
3,Rowenta RO2727EA City Space Cyclonic Aspirador...,"59,99 €",513 valoraciones,NaN,750 vatios,"47 x 28,5 x 30 cm",1.2 litros
4,Rowenta Compact Power Cyclonic RO3753EA Aspira...,"84,99 €",5.257 valoraciones,NaN,750 vatios,"40,9 x 27,6 x 29 cm",1.5 litros


###### Arreglando el valor del precio

In [39]:
try:
    # Se quita los € por vacío
    aspiradoras["Precio"] = aspiradoras["Precio"].str.replace("€","");
    # Se reemplaza la coma por punto
    aspiradoras["Precio"] = aspiradoras["Precio"].str.replace(",",".");
    #Se convierte a float
    aspiradoras["Precio"] = aspiradoras["Precio"].astype(float);    
    
except Exception as e: 
    print(e)
    


Can only use .str accessor with string values, which use np.object_ dtype in pandas


###### Arreglando el valoraciones

In [40]:
try:
    # Se retira el texto valoraciones
    aspiradoras['Opiniones'] = aspiradoras['Opiniones'].str.replace('valoraciones','');
    aspiradoras['Opiniones'] = aspiradoras['Opiniones'].str.replace('valoración','');
    # Se quita el punto
    aspiradoras['Opiniones'] = aspiradoras['Opiniones'].str.replace('.','');
    # Se cambia el tipo de dato a int 
    aspiradoras['Opiniones'] = aspiradoras['Opiniones'].astype(int);
    
except Exception as e: 
    print(e)
   


C:\Users\xgutierrez\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\xgutierrez\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\xgutierrez\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to

In [42]:
aspiradoras.head()

,Nombre Aspiradora,Precio,Opiniones,Peso,Potencia,Dimensiones del producto,Volumen
0,AmazonBasics - Aspiradora cilíndrica sin bolsa...,49.99,3934,NaN,700 vatios,"36,3 x 29,4 x 24,7 cm",1.5 litros
1,"INSE Aspiradora Escopa sin Cable, Vertical y d...",199.00,49,NaN,250 vatios,"34,5 x 14 x 24 cm",NaN
2,"Aspiradora Escoba, MEIYOU Aspirador Mano con C...",59.96,354,NaN,600 vatios,NaN,NaN
3,Rowenta RO2727EA City Space Cyclonic Aspirador...,59.99,513,NaN,750 vatios,"47 x 28,5 x 30 cm",1.2 litros
4,Rowenta Compact Power Cyclonic RO3753EA Aspira...,84.99,5257,NaN,750 vatios,"40,9 x 27,6 x 29 cm",1.5 litros


###### Arreglando el Peso

In [45]:
try:    
    aspiradoras['Peso'] = aspiradoras['Peso'].str.replace('kilogramos','')
    aspiradoras['Peso'] = aspiradoras['Peso'].astype(float)

    #Cambiando el nombre de una columna
    aspiradoras.rename(columns={'Peso':'Peso(Kg)'},inplace=True)
    
except Exception as e: 
    print(e)
   

'Peso'


In [46]:
aspiradoras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 7 columns):
Nombre Aspiradora           211 non-null object
Precio                      211 non-null float64
Opiniones                   211 non-null int32
Peso(Kg)                    11 non-null float64
Potencia                    154 non-null object
Dimensiones del producto    162 non-null object
Volumen                     119 non-null object
dtypes: float64(2), int32(1), object(4)
memory usage: 10.8+ KB


###### Arreglando la potencia

In [87]:
try:    
    aspiradoras['Potencia'] = aspiradoras['Potencia'].str.replace('vatios','')
    aspiradoras['Potencia'] = aspiradoras['Potencia'].str.replace('voltios','')
    aspiradoras['Potencia'] = aspiradoras['Potencia'].str.replace('100 kW','100000')
    aspiradoras['Potencia'] = aspiradoras['Potencia'].str.replace('watt_hours','')
    aspiradoras['Potencia'] = aspiradoras['Potencia'].astype(float)

    #Cambiando el nombre de una columna
    aspiradoras.rename(columns={'Potencia':'Potencia(vatios)'},inplace=True)
    
except Exception as e: 
    print(e)
   


Can only use .str accessor with string values, which use np.object_ dtype in pandas


In [90]:
# Buscando todos los registros que tienen la palabra kw
#aspiradoras[aspiradoras['Potencia'].notnull() & aspiradoras['Potencia'].str.contains('kW')]
#aspiradoras[aspiradoras['Potencia'].notnull() & aspiradoras['Potencia'].str.contains('watt_hours')]

In [89]:
aspiradoras.head()

,Nombre Aspiradora,Precio,Opiniones,Peso(Kg),Potencia,Dimensiones del producto,Volumen
0,AmazonBasics - Aspiradora cilíndrica sin bolsa...,49.99,3934,NaN,700.0,"36,3 x 29,4 x 24,7 cm",1.5 litros
1,"INSE Aspiradora Escopa sin Cable, Vertical y d...",199.00,49,NaN,250.0,"34,5 x 14 x 24 cm",NaN
2,"Aspiradora Escoba, MEIYOU Aspirador Mano con C...",59.96,354,NaN,600.0,NaN,NaN
3,Rowenta RO2727EA City Space Cyclonic Aspirador...,59.99,513,NaN,750.0,"47 x 28,5 x 30 cm",1.2 litros
4,Rowenta Compact Power Cyclonic RO3753EA Aspira...,84.99,5257,NaN,750.0,"40,9 x 27,6 x 29 cm",1.5 litros
